# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,usinsk,65.9939,57.5281,-25.52,100,98,1.79,RU,1702166527
1,1,bilibino,68.0546,166.4372,-17.52,99,99,0.94,RU,1702166527
2,2,ilulissat,69.2167,-51.1000,-5.99,73,75,5.66,GL,1702166527
3,3,badger,64.8000,-147.5333,-22.58,79,29,1.25,US,1702166527
4,4,krasnovishersk,60.4078,57.0820,-31.68,99,16,0.81,RU,1702166528


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:



# Create the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols=['City', 'Humidity'],
    width=800,
    height=500,
    title='City Humidity Map')

# Display the map plot_1
map_plot_1



### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 36) &  
    (city_data_df['Humidity'] >= 30) & (city_data_df['Humidity'] <= 60)  
)

ideal_weather_df = city_data_df[ideal_conditions]

# Display the resulting DataFrame
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,88,port hedland,-20.3167,118.5667,29.03,32,23,7.72,AU,1702166286
107,107,espinosa,-14.9261,-42.8192,28.67,40,99,2.36,BR,1702166551
139,139,salto,-31.3833,-57.9667,23.26,58,42,1.79,UY,1702166547
164,164,veraval,20.9000,70.3667,21.31,44,0,5.21,IN,1702166524
188,188,kone,-21.0595,164.8658,29.04,42,97,3.85,NC,1702166567


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
88,port hedland,AU,-20.3167,118.5667,32,
107,espinosa,BR,-14.9261,-42.8192,40,
139,salto,UY,-31.3833,-57.9667,58,
164,veraval,IN,20.9000,70.3667,44,
188,kone,NC,-21.0595,164.8658,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'type': 'accommodation.hotel',  
    'radius': radius,  
    'limit': 50}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as e:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port hedland - nearest hotel: The Esplanade Hotel
espinosa - nearest hotel: No hotel found
salto - nearest hotel: Hotel Español Salto
veraval - nearest hotel: Shyam
kone - nearest hotel: Pacifik Appartels
mareeba - nearest hotel: The Gateway Hotel
mount isa - nearest hotel: Ibis Styles
east ballina - nearest hotel: Ballina Beach Resort
opuwo - nearest hotel: Ohakane Lodge
padre bernardo - nearest hotel: Hotel Fazenda Mestre D'Armas
cabo san lucas - nearest hotel: Comfort Rooms
boali - nearest hotel: No hotel found
al khafji - nearest hotel: Al Muhaidb Residence
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
sur - nearest hotel: Sur Hotel
quillacollo - nearest hotel: Casa Campestre
charters towers - nearest hotel: No hotel found
orkney - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
marsa alam - nearest hotel: Blue House Hotel
isla vista - nearest hotel: The Club & Guest House
santa cruz - nearest hotel: Aviador Hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
88,port hedland,AU,-20.3167,118.5667,32,The Esplanade Hotel
107,espinosa,BR,-14.9261,-42.8192,40,No hotel found
139,salto,UY,-31.3833,-57.9667,58,Hotel Español Salto
164,veraval,IN,20.9000,70.3667,44,Shyam
188,kone,NC,-21.0595,164.8658,42,Pacifik Appartels
200,mareeba,AU,-17.0000,145.4333,58,The Gateway Hotel
214,mount isa,AU,-20.7333,139.5000,41,Ibis Styles
237,east ballina,AU,-28.8667,153.5833,60,Ballina Beach Resort
273,opuwo,NaN,-18.0607,13.8400,37,Ohakane Lodge
274,padre bernardo,BR,-15.3500,-48.5000,43,Hotel Fazenda Mestre D'Armas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
#create map
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM"
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    width=800,
    height=500,
    title='City Humidity Map with Hotel Information')

# Display the map plot_1
map_plot_2